In [42]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV,train_test_split,StratifiedKFold
from sklearn.svm  import SVC
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, balanced_accuracy_score,f1_score, classification_report, confusion_matrix)
import warnings
# Silence all FutureWarnings
warnings.filterwarnings("ignore")

In [286]:
df = pd.read_csv("Data/cirrhosis.csv")
df.drop(df.columns[0],axis=1,inplace=True)
df

,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,400,D,D-penicillamine,21464,F,Y,Y,Y,Y,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,4.0
1,4500,C,D-penicillamine,20617,F,N,Y,Y,N,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,3.0
2,1012,D,D-penicillamine,25594,M,N,N,N,S,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0,4.0
3,1925,D,D-penicillamine,19994,F,N,Y,Y,S,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,4.0
4,1504,CL,Placebo,13918,F,N,Y,Y,N,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,681,D,NaN,24472,F,NaN,NaN,NaN,N,1.2,NaN,2.96,NaN,NaN,NaN,NaN,174.0,10.9,3.0
414,1103,C,NaN,14245,F,NaN,NaN,NaN,N,0.9,NaN,3.83,NaN,NaN,NaN,NaN,180.0,11.2,4.0
415,1055,C,NaN,20819,F,NaN,NaN,NaN,N,1.6,NaN,3.42,NaN,NaN,NaN,NaN,143.0,9.9,3.0
416,691,C,NaN,21185,F,NaN,NaN,NaN,N,0.8,NaN,3.75,NaN,NaN,NaN,NaN,269.0,10.4,3.0


In [287]:
df1 = df[0:312]
# df[df.isnull().any(axis=1)]
df = df[df['Stage'].isnull()==False]
df

,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,400,D,D-penicillamine,21464,F,Y,Y,Y,Y,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,4.0
1,4500,C,D-penicillamine,20617,F,N,Y,Y,N,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,3.0
2,1012,D,D-penicillamine,25594,M,N,N,N,S,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0,4.0
3,1925,D,D-penicillamine,19994,F,N,Y,Y,S,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,4.0
4,1504,CL,Placebo,13918,F,N,Y,Y,N,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,681,D,NaN,24472,F,NaN,NaN,NaN,N,1.2,NaN,2.96,NaN,NaN,NaN,NaN,174.0,10.9,3.0
414,1103,C,NaN,14245,F,NaN,NaN,NaN,N,0.9,NaN,3.83,NaN,NaN,NaN,NaN,180.0,11.2,4.0
415,1055,C,NaN,20819,F,NaN,NaN,NaN,N,1.6,NaN,3.42,NaN,NaN,NaN,NaN,143.0,9.9,3.0
416,691,C,NaN,21185,F,NaN,NaN,NaN,N,0.8,NaN,3.75,NaN,NaN,NaN,NaN,269.0,10.4,3.0


In [288]:
# df1['Copper'].fillna((df1['Copper'].mean()),inplace=True)
df1.loc[df1['Copper'].isnull(),'Copper'] = df['Copper'].mean()
# df['Cholesterol_Trygli_Nan'] = df1['Cholesterol'].isnull() & df1['Tryglicerides'].isnull()
# df1.loc[df1['Cholesterol'].isnull() & df1['Tryglicerides'].isnull(),['Cholesterol','Tryglicerides']] = 0 

# df1.loc[df1['Tryglicerides'].isnull(),['Tryglicerides']] = df1['Tryglicerides'].mean()

df1.loc[df1['Platelets'].isnull(),'Platelets'] = df['Platelets'].mean()
df1.isnull().any()

N_Days           False
Status           False
Drug             False
Age              False
Sex              False
Ascites          False
Hepatomegaly     False
Spiders          False
Edema            False
Bilirubin        False
Cholesterol       True
Albumin          False
Copper           False
Alk_Phos         False
SGOT             False
Tryglicerides     True
Platelets        False
Prothrombin      False
Stage            False
dtype: bool

In [289]:
df1.dtypes.value_counts()

float64    10
object      7
int64       2
Name: count, dtype: int64

In [290]:
# df['Patient_participated'] = 1
# df.loc[312:,['Patient_participated']] = 0
df2 = df[312:]

fill_value={}
# Define specific fill values for each column type
for i in df2.columns:
    if i not in ['Cholesterol','Tryglicerides']:
        if df2[i].dtype in [float,int]:
            fill_value[i] = 0
        elif df2[i].dtype == object:
            fill_value[i] = 'Missing'

df2.fillna(fill_value, inplace=True)
df2



,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
313,3561,D,Missing,23741,F,Missing,Missing,Missing,S,1.4,NaN,3.04,0.0,0.0,0.0,NaN,331.0,12.1,4.0
314,2844,C,Missing,19724,F,Missing,Missing,Missing,N,0.7,NaN,4.03,0.0,0.0,0.0,NaN,226.0,9.8,4.0
315,2071,D,Missing,27394,F,Missing,Missing,Missing,S,0.7,NaN,3.96,0.0,0.0,0.0,NaN,0.0,11.3,4.0
317,1680,C,Missing,15706,F,Missing,Missing,Missing,N,0.7,NaN,3.68,0.0,0.0,0.0,NaN,306.0,9.5,2.0
319,2403,C,Missing,16071,F,Missing,Missing,Missing,S,0.4,NaN,3.81,0.0,0.0,0.0,NaN,226.0,10.5,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,681,D,Missing,24472,F,Missing,Missing,Missing,N,1.2,NaN,2.96,0.0,0.0,0.0,NaN,174.0,10.9,3.0
414,1103,C,Missing,14245,F,Missing,Missing,Missing,N,0.9,NaN,3.83,0.0,0.0,0.0,NaN,180.0,11.2,4.0
415,1055,C,Missing,20819,F,Missing,Missing,Missing,N,1.6,NaN,3.42,0.0,0.0,0.0,NaN,143.0,9.9,3.0
416,691,C,Missing,21185,F,Missing,Missing,Missing,N,0.8,NaN,3.75,0.0,0.0,0.0,NaN,269.0,10.4,3.0


In [291]:
df3 = pd.concat([df1,df2],axis=0)
df3['Cholesterol_Trygli_Nan'] = True
df3.loc[df3['Cholesterol'].isnull() & df3['Tryglicerides'].isnull(),'Cholesterol_Trygli_Nan'] = False
df3.loc[df3['Cholesterol'].isnull() & df3['Tryglicerides'].isnull(),['Cholesterol','Tryglicerides']] = 0
df3['Tryglicerides'].fillna(df3['Tryglicerides'].mean(),inplace = True)


df3['Patient_participated'] = True
df3.loc[312:,['Patient_participated']] = False

df3[['Cholesterol','Tryglicerides','Cholesterol_Trygli_Nan']]


,Cholesterol,Tryglicerides,Cholesterol_Trygli_Nan
0,261.0,172.0,True
1,302.0,88.0,True
2,176.0,55.0,True
3,244.0,92.0,True
4,279.0,72.0,True
...,...,...,...
413,0.0,0.0,False
414,0.0,0.0,False
415,0.0,0.0,False
416,0.0,0.0,False


In [303]:
# df[df.columns[2]].fillna("Other",inplace=True)
# for i in df.columns:
#     if df[i].isna().any()==True:
#         df.fillna({i: df[i].mean()},inplace=True)  #I have written this line knowing that categorical variables no longer have any NAN values
df3["Age"] = round(df["Age"]/365)
df3['Stage'].value_counts()

Stage
3.0    155
4.0    144
2.0     92
1.0     21
Name: count, dtype: int64

# **As we can see from above , this is a highly imbalanced dataset**  #

In [293]:
# lists = [
# df3.groupby("Stage")["N_Days"].mean(),
# df3.groupby("Stage")["Age"].mean(),
# df3.groupby("Stage")[df.columns[9]].mean(),
# df3.groupby("Stage")[df.columns[10]].mean(),
# df3.groupby("Stage")[df.columns[11]].mean(),
# df3.groupby("Stage")[df.columns[12]].mean(),
# df3.groupby("Stage")[df.columns[13]].mean(),
# df3.groupby("Stage")[df.columns[14]].mean(),
# df3.groupby("Stage")[df.columns[15]].mean(),
# df3.groupby("Stage")[df.columns[16]].mean(),
# df3.groupby("Stage")[df.columns[17]].mean()]

lists = [df3.groupby("Stage")[i].mean() for i in df3.columns if df3[i].dtype != 'object']
lists
print(pd.concat(lists,axis=1))
#Platelets,Age,tryglycerides,SGOT,alkaline phosphatase,Copper,albumin,Bilirubin,prothrombin,N_Days

            N_Days        Age  Bilirubin  Cholesterol   Albumin     Copper  \
Stage                                                                        
1.0    2654.809524  47.000000   1.361905   165.809524  3.705238  47.857143   
2.0    2389.836957  49.478261   2.453261   234.173913  3.607065  49.865743   
3.0    1997.451613  48.974194   2.825806   308.535484  3.592387  71.283871   
4.0    1420.250000  53.812500   4.427083   222.861111  3.302431  96.004503   

          Alk_Phos       SGOT  Tryglicerides   Platelets  Prothrombin  Stage  \
Stage                                                                          
1.0    1290.828571  64.350476      56.238095  290.100107    10.266667    1.0   
2.0    1323.182609  84.159565      75.554348  270.968503    10.525000    2.0   
3.0    1611.970323  96.252968      96.416397  263.027126    10.469032    3.0   
4.0    1527.034722  98.780069      85.145833  221.743056    11.070139    4.0   

       Cholesterol_Trygli_Nan  Patient_participate

In [294]:
# print(pd.crosstab(df3["Drug"],df3["Stage"],normalize='index') ,'\n')
# print(pd.crosstab(df3["Sex"],df3["Stage"],normalize='index'),'\n' )
# print(pd.crosstab(df["Status"],df3["Stage"],normalize='index'),'\n' )
# print(pd.crosstab(df3[df3.columns[5]],df3["Stage"],normalize='index'),'\n')
# print(pd.crosstab(df3[df3.columns[6]],df3["Stage"],normalize='index'),'\n' )
# print(pd.crosstab(df3[df3.columns[7]],df3["Stage"],normalize='index'),'\n' )
# print(pd.crosstab(df3[df3.columns[8]],df3["Stage"],normalize='index'),'\n' )

for i in df3.columns:
    if df3[i].dtype == 'object':
        print(pd.crosstab(df3[i],df3["Stage"],normalize='index'),'\n')
# Status,Ascites,Hepatomehgaly,Spiders,Edema
#Margins = True for displaying row and column total 
#normalize = 'index' for showing row relative frequencies

Stage        1.0       2.0       3.0       4.0
Status                                        
C       0.082609  0.278261  0.421739  0.217391
CL      0.000000  0.200000  0.400000  0.400000
D       0.012739  0.146497  0.305732  0.535032 

Stage                 1.0       2.0       3.0       4.0
Drug                                                   
D-penicillamine  0.075949  0.221519  0.354430  0.348101
Missing          0.050000  0.250000  0.350000  0.350000
Placebo          0.025974  0.207792  0.415584  0.350649 

Stage       1.0       2.0       3.0       4.0
Sex                                          
F      0.048913  0.228261  0.377717  0.345109
M      0.068182  0.181818  0.363636  0.386364 

Stage         1.0       2.0       3.0       4.0
Ascites                                        
Missing  0.050000  0.250000  0.350000  0.350000
N        0.055556  0.225694  0.413194  0.305556
Y        0.000000  0.083333  0.041667  0.875000 

Stage              1.0       2.0       3.0       4.0


In [253]:
df3.dtypes

N_Days                      int64
Status                     object
Drug                       object
Age                       float64
Sex                        object
Ascites                    object
Hepatomegaly               object
Spiders                    object
Edema                      object
Bilirubin                 float64
Cholesterol               float64
Albumin                   float64
Copper                    float64
Alk_Phos                  float64
SGOT                      float64
Tryglicerides             float64
Platelets                 float64
Prothrombin               float64
Stage                     float64
Cholesterol_Trygli_Nan       bool
Patient_participated         bool
dtype: object

In [295]:
"""While StandardScaler can be a precursor to methods that might leverage standardized data for outlier identification (like Z-score thresholding), 
it is not a method for outlier detection in itself and is sensitive to the very outliers one might be trying to detect. """

df4 = df3.select_dtypes(["int","float"]).drop("Stage",axis=1)
# min,max = [],[]
# for i in df2.columns:
#     min.append(df2[i].quantile(0.001))
#     max.append(df2[i].quantile(0.999))
mins = df4.quantile(0.001)
maxs = df4.quantile(0.999)
mask = ((df4[df4.columns] > mins ) & (df4[df4.columns] < maxs)).all(axis=1)
df4 = df4.loc[mask]
df4
   #We can use both df.loc[..] and df[condition] but loc[] is preferred for boolean indexing




,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin
0,400,59.0,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2
1,4500,56.0,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6
2,1012,70.0,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0
3,1925,55.0,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3
4,1504,38.0,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9
...,...,...,...,...,...,...,...,...,...,...,...
307,1153,61.0,0.4,246.0,3.58,24.0,797.0,91.00,113.0,288.0,10.4
308,994,58.0,0.4,260.0,2.75,41.0,1166.0,70.00,82.0,231.0,10.8
309,939,62.0,1.7,434.0,3.35,39.0,1713.0,171.00,100.0,234.0,10.2
310,839,38.0,2.0,247.0,3.16,69.0,1050.0,117.00,88.0,335.0,10.5


In [296]:
df3.select_dtypes(["object"])

,Status,Drug,Sex,Ascites,Hepatomegaly,Spiders,Edema
0,D,D-penicillamine,F,Y,Y,Y,Y
1,C,D-penicillamine,F,N,Y,Y,N
2,D,D-penicillamine,M,N,N,N,S
3,D,D-penicillamine,F,N,Y,Y,S
4,CL,Placebo,F,N,Y,Y,N
...,...,...,...,...,...,...,...
413,D,Missing,F,Missing,Missing,Missing,N
414,C,Missing,F,Missing,Missing,Missing,N
415,C,Missing,F,Missing,Missing,Missing,N
416,C,Missing,F,Missing,Missing,Missing,N


In [297]:
dummies = pd.get_dummies(df3.select_dtypes(["object"]), drop_first=True).loc[df3.index]
X = pd.concat([dummies, df3[df4.columns]], axis=1)
Y = df3["Stage"].loc[df3.index]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

numeric_cols = df4.columns.tolist()   # order preserved from df2
cat_cols = [c for c in X.columns if c not in numeric_cols]   #There is no String in X since all converted to dummies

st = StandardScaler()

X_train_num = pd.DataFrame(st.fit_transform(X_train[numeric_cols]), columns=numeric_cols, index=X_train.index)  
X_test_num  = pd.DataFrame(st.transform(X_test[numeric_cols]),  columns=numeric_cols, index=X_test.index)

"""We are standardizing the training and testing data using statistical values of training data only. So, the statistics of test data remains hidden making it unbiased."""

# Joining categorical and numerical columns and indexing them appropriately for both test and train data
X_train_scaled = pd.concat([X_train[cat_cols].reset_index(drop=True),X_train_num.reset_index(drop=True)], axis=1)
X_train_scaled.index = X_train.index  # restore original index
X_test_scaled = pd.concat([X_test[cat_cols].reset_index(drop=True),X_test_num.reset_index(drop=True)], axis=1)
X_test_scaled.index = X_test.index

# a = [7,8,10,15]
# # df3 = pd.DataFrame(st.fit_transform(df2),columns=df2.columns,index=df2.index)
# # mask = (np.abs(df3[df3.columns[a]]) < 3*(df3[df3.columns[a]].std())).all(axis = 1)  

# mask = (np.abs(X_train_scaled[X_train_scaled.columns[a]]) < 3 * X_train_scaled[X_train_scaled.columns[a]].std()).all(axis=1)

# X_train_final = X_train_scaled.loc[mask]
# y_train_final = y_train.loc[X_train_final.index]
"""df3.columns[a] → selects only the columns at positions 0, 1, 3, 8 in df3.
(np.abs(... ) < 3*(...)) → produces a Boolean DataFrame for those 4 columns only.
.all(axis=1) → for each row, checks if all 4 of those selected columns satisfy the condition."""

# plt.hist(X_train_scaled[X_train_scaled.columns[10]])
# 7,8,10,15 columns are normally distributed
# df3 = df3.loc[mask]
X_train_scaled.dtypes.value_counts()
X_train_scaled.isnull().any()

Status_CL         False
Status_D          False
Drug_Missing      False
Drug_Placebo      False
Sex_M             False
Ascites_N         False
Ascites_Y         False
Hepatomegaly_N    False
Hepatomegaly_Y    False
Spiders_N         False
Spiders_Y         False
Edema_S           False
Edema_Y           False
N_Days            False
Age               False
Bilirubin         False
Cholesterol       False
Albumin           False
Copper            False
Alk_Phos          False
SGOT              False
Tryglicerides     False
Platelets         False
Prothrombin       False
dtype: bool

In [304]:
models = {"LogisticRegression":{"model": LogisticRegression(solver="saga", max_iter=5000, random_state=36,class_weight='balanced'),
                                "param":{"C":[0.01,0.1,1.0,5.0],"penalty":['l1','l2']}},
         "SupportVector"      :{"model":SVC(gamma="scale",probability=False,class_weight='balanced'),
                                "param":{"kernel":['linear','rbf'],"C":[.1,1,10]}},
         "RandomForest"       :{"model":RandomForestClassifier(class_weight='balanced'),
                                "param":{"n_estimators":[100,200]}},
         "DecisionTree"       :{"model":DecisionTreeClassifier(),
                                "param":{}},
         
}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring_for_grid = "balanced_accuracy"   # better for class imbalance

b=[]
for i,j in models.items():
    grid = GridSearchCV(j["model"],param_grid=j["param"],cv=cv,return_train_score=False)
    grid.fit(X_train_scaled,y_train)
    best = grid.best_estimator_
    y_pred = best.predict(X_test_scaled)

    acc = accuracy_score(y_test, y_pred)
    bal_acc = balanced_accuracy_score(y_test, y_pred)
    f1_macro = f1_score(y_test, y_pred, average="macro")

    b.append({
        "model":i,
        "Best_params":grid.best_params_,
        "Score": grid.best_score_,
        "test_accuracy": acc,
        "test_balanced_accuracy": bal_acc,
        "test_f1_macro": f1_macro
    })




k = pd.DataFrame(b).sort_values("test_balanced_accuracy", ascending=False)
k


,model,Best_params,Score,test_accuracy,test_balanced_accuracy,test_f1_macro
2,RandomForest,{'n_estimators': 200},0.528811,0.530120,0.400503,0.396804
1,SupportVector,"{'C': 10, 'kernel': 'rbf'}",0.498415,0.445783,0.393394,0.391661
3,DecisionTree,{},0.407040,0.433735,0.358117,0.342687
0,LogisticRegression,"{'C': 5.0, 'penalty': 'l2'}",0.458928,0.325301,0.338937,0.278274


In [305]:
# 1. Find best model by balanced accuracy (or whichever metric you want)
best_row = k.loc[k["test_balanced_accuracy"].idxmax()]
print("Best model:", best_row["model"])

# 2. Refit that model on training set to get estimator
best_model = models[best_row["model"]]["model"].set_params(**best_row["Best_params"])
best_model.fit(X_train_scaled, y_train)

# 3. Predictions
y_pred = best_model.predict(X_test_scaled)

# 4. Reports
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Best model: RandomForest

Classification Report:
               precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         4
         2.0       0.35      0.37      0.36        19
         3.0       0.47      0.55      0.51        31
         4.0       0.74      0.69      0.71        29

    accuracy                           0.53        83
   macro avg       0.39      0.40      0.40        83
weighted avg       0.52      0.53      0.52        83


Confusion Matrix:
 [[ 0  1  3  0]
 [ 0  7 10  2]
 [ 0  9 17  5]
 [ 0  3  6 20]]


In [307]:
from sklearn.ensemble import GradientBoostingClassifier

# Define the GradientBoostingClassifier model and parameters
gbc_model = GradientBoostingClassifier(random_state=42)
gbc_params = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5]
}

# Perform GridSearchCV
grid_gbc = GridSearchCV(gbc_model, param_grid=gbc_params, cv=cv, scoring=scoring_for_grid, return_train_score=False)
grid_gbc.fit(X_train_scaled, y_train)

# Get the best estimator and make predictions
best_gbc = grid_gbc.best_estimator_
y_pred_gbc = best_gbc.predict(X_test_scaled)

# Calculate and print metrics
acc_gbc = accuracy_score(y_test, y_pred_gbc)
bal_acc_gbc = balanced_accuracy_score(y_test, y_pred_gbc)
f1_macro_gbc = f1_score(y_test, y_pred_gbc, average="macro")

print("Gradient Boosting Classifier Metrics:")
print(f"Best Parameters: {grid_gbc.best_params_}")
print(f"Best Cross-Validation Score: {grid_gbc.best_score_:.4f}")
print(f"Test Accuracy: {acc_gbc:.4f}")
print(f"Test Balanced Accuracy: {bal_acc_gbc:.4f}")
print(f"Test F1 Macro Score: {f1_macro_gbc:.4f}")

print("\nClassification Report:\n", classification_report(y_test, y_pred_gbc))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_gbc))

Gradient Boosting Classifier Metrics:
Best Parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}
Best Cross-Validation Score: 0.3675
Test Accuracy: 0.5422
Test Balanced Accuracy: 0.3989
Test F1 Macro Score: 0.3927

Classification Report:
               precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         4
         2.0       0.62      0.26      0.37        19
         3.0       0.46      0.68      0.55        31
         4.0       0.66      0.66      0.66        29

    accuracy                           0.54        83
   macro avg       0.43      0.40      0.39        83
weighted avg       0.54      0.54      0.52        83


Confusion Matrix:
 [[ 0  1  3  0]
 [ 0  5 12  2]
 [ 0  2 21  8]
 [ 0  0 10 19]]


### Reasoning for Low Accuracy Without SMOTE

Your model's accuracy, while comparable or even better than some other attempts without SMOTE, is still relatively low primarily due to the **highly imbalanced nature of the dataset** for the 'Stage' target variable.

Here's a breakdown of the key factors:

1.  **Class Imbalance**: As observed earlier (e.g., from `df3['Stage'].value_counts()`), the distribution of samples across different 'Stage' classes is very uneven. Some stages have significantly fewer examples than others. In such scenarios, a model tends to become biased towards the majority classes, as correctly predicting them often yields a higher overall accuracy score. This comes at the cost of poor performance (low precision and recall) on minority classes.

2.  **Limitations of `class_weight='balanced'`**: While setting `class_weight='balanced'` in models like Logistic Regression, SVM, and Random Forest is a good step, it's a passive technique. It tells the model to pay more attention to misclassifications of minority classes during training by adjusting the cost function. However, it doesn't change the underlying data distribution. The model still sees the same skewed number of samples per class.

3.  **No Synthetic Data Generation**: Techniques like SMOTE (Synthetic Minority Over-sampling Technique) actively create synthetic samples for the minority classes, effectively balancing the dataset. This provides the model with more examples of the underrepresented classes to learn from, leading to a more robust decision boundary and significantly better performance, especially in terms of balanced accuracy and recall for minority classes.